In [ ]:
from Library import *

In [ ]:
df = pd.read_csv("../translated_data.csv")
tiket_df = pd.DataFrame(df)

In [ ]:
import pandas as pd

# Pastikan Anda sudah memuat DataFrame bernama tiket_df
# Jika Anda belum memuatnya, ganti baris di bawah ini dengan kode pemuatan data Anda.
# Contoh: tiket_df = pd.read_csv("path/to/translated_data.csv")

# =========================================================
# === MOHON GANTI NAMA KOLOM DI BAWAH SESUAI DATA ANDA ===
# =========================================================

# Kolom yang berisi teks/body tiket
TEXT_COLUMN = "body"

# Daftar kolom yang merupakan target klasifikasi multi-output Anda (misalnya Kategori, Prioritas)
TARGET_COLUMNS = ["type", "queue", "priority"]
# =========================================================

if tiket_df.empty:
    print("Error: DataFrame 'tiket_df' kosong atau belum dimuat.")
else:
    # 1. Menghitung Jumlah Kata (Word Count) untuk Setiap Baris
    # Menggunakan .astype(str) untuk menghindari error jika ada nilai non-string (misalnya NaN)
    print("Langkah 1: Menghitung jumlah kata...")
    tiket_df["word_count"] = (
        tiket_df[TEXT_COLUMN].astype(str).apply(lambda x: len(x.split()))
    )

    p95_results = {}

    # 2. Iterasi untuk Menghitung P95 di Setiap Kolom Target
    print("\nLangkah 2: Menghitung P95 Word Length per Kelas Target...")
    for target in TARGET_COLUMNS:
        if target in tiket_df.columns and not tiket_df[target].isnull().all():

            # Menghitung P95 (Quantile 0.95) dari word_count yang dikelompokkan berdasarkan kelas target
            p95_series = (
                tiket_df.groupby(target)["word_count"]
                .quantile(0.95)
                .sort_values(ascending=False)
            )
            p95_results[target] = p95_series

            print(f"\n--- P95 Word Length untuk Target: '{target}' ---")
            print(p95_series.to_string())
            print("-" * 50)
        else:
            print(f"Peringatan: Kolom target '{target}' tidak ditemukan atau kosong.")

    # 3. Menentukan Max Length Rekomendasi
    # Cari nilai P95 tertinggi dari semua kelas di semua target
    max_p95 = 0
    if p95_results:
        all_p95_values = pd.concat(p95_results.values())
        max_p95 = all_p95_values.max()

    # Faktor konversi (Word ke Token: ~1.3) dan Margin (P95 ke P99)
    # Kita gunakan faktor 1.5 untuk margin keamanan yang lebih besar (Word -> Token + Buffer)
    RECOMMENDED_MAX_LENGTH = int(max_p95 * 1.5) if max_p95 > 0 else 128

    print("\n" + "=" * 50)
    print("               REKOMENDASI MAX_LENGTH")
    print("=" * 50)
    print(f"Nilai P95 (Word Count) Tertinggi Ditemukan: {int(max_p95)} kata")
    print(
        f"Rekomendasi Max Length (Token) untuk BERT (P95 * 1.5 buffer): {RECOMMENDED_MAX_LENGTH} token"
    )
    print(
        "\nAnda harus menggunakan nilai ini (atau mendekatinya, mis. 256 atau 512) saat tokenisasi."
    )

In [ ]:
from transformers import AutoTokenizer

MODEL_NAME = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# your chosen max length (based on average words per ticket)
encodings = tokenizer(
    tiket_df["translated_body"].astype(str).tolist(),  # list of texts to encode
    padding=True,  # pad to max_length within the batch
    truncation=True,  # truncate longer sequences
    max_length=256,  # ✅ your parameter here
    return_tensors="pt",
)
display(encodings)